In [ ]:
# 위치 확인
import os
# print(os.getcwd())

In [ ]:
# 파일 불러오기
# 전처리
f = open("input.txt", 'r')
line = f.readlines()

List = []
List_2 = []
for i in range(len(line)):
    line[i] = line[i].replace('\n','')
    line[i]= line[i].split('\t')
    List.append(set(list(map(int, line[i]))))

In [ ]:
class Algorithm:
    def __init__(self, m):
        self.Min_sup = m * 0.01
        self.dblen = 0
        self.k = 0

    # n개의 파티션으로 나누기
    def Partition_Database(self, db, k):
        self.dblen = len(db)
        self.k = k
        count= int(len(db)/k)
        
        new_db = [db[i:i + count]
                  for i in range(0, len(db), count)]
        
        return new_db

    #각 분할에 한정된 빈발항목 집합을 발견 (scan 1)
    def Scan_one(self, new_db):
        P_l = []
        total = self.dblen
        
        for P in new_db:
            local_Min_sup = self.Min_sup / self.k
    #                     = len(P)/total
            L_p = self.Apriori(P, local_Min_sup)
            P_l.append(L_p)
            
        return P_l
    
    # 
    def Apriori(self, db, Min_sup):
        
        # L_1 구하기
        L_1 = []
        total = len(db)
        freq_dict = {}
        
        # 빈도수 count
        for i in db:
            for j in i:
                if j not in freq_dict.keys():
                    freq_dict[j] = 0
                freq_dict[j] += 1
        
        # Min_sup 보다 큰 Pattern 찾기
        for i,j in freq_dict.items():
            if j/total >= Min_sup:
                L_1.append([i])
                
        U = []
        U.append(L_1)
        i = 0
        
        while True:
            Candidate_k = self.Self_joining(U[i], i)
            Freq_itemset_k = self.Pruning(Candidate_k, U[i])
            L_k = self.Scan(Freq_itemset_k, db)
            item = self.Freq_itemset(L_k, Freq_itemset_k, Min_sup)
            
            if len(item) == 0:
                break
            else:
                U.append(item)
            i = i+1
        
        return U
    
    def Self_joining(self, db, length):
        Candidate = []
        
        for i in db:
            for j in db:
                if i != j:
                    unit_1 = set(i)
                    unit_2 = set(j)
                    join = unit_1.union(unit_2)
                    if len(join) == length + 2:
                        C = join
                        if C not in Candidate:
                            Candidate.append(C)
        
        return Candidate
    
    def Pruning(self, C_k, db):
        
        for i in C_k:
            for j in i:
                temp = i.remove(j)
                if temp not in db:
                    C_k.remove(i)
                    break
        
        return C_k
     
    def Scan(self, Freq_pattern, db):
        dic = {}
        
        for i in Freq_pattern:
            temp = 0
            for j in range(len(db)):
                if i.issubset(db[j]) == True:
                    temp = temp + 1
            item = Freq_pattern.index(i)
            dic[item] = temp
        
        return dic
    
    def Freq_itemset(self, Freq_dic, candidate, Min_sup):
        temp = []
        total = self.dblen/self.k
        
        for i, j in Freq_dic.items():
            if j/total >= Min_sup:
                temp.append(list(candidate[i]))
                
        return temp
    
    def Consolidate(self, Partition_Freq):
        C = []
        
        for i in Partition_Freq:
            for j in i:
                for z in j:
                    zz = set(z)
                    if zz not in C:
                        C.append(zz)
        return C
    
    def Scan_two(self, Candidate, transection_db):
        dic = {}
        L = []
        
        for i in Candidate:
            temp = 0
            for j in transection_db:
                if i.issubset(j) == True:
                    temp = temp + 1
            item = Candidate.index(i)
            dic[item] = temp
            
        total = self.dblen
        
        for i,j in dic.items():
            if j/total >= self.Min_sup:
                L.append(Candidate[i])
        
        return L
    
    def association_rule(self, h):
        L = []
        
        for i in range(len(h)):
            if len(h[i]) != 1:
                m = self.make_set(h[i])
                for j in m:
                    k = h[i] - j
                    temp = [j, k]
                    L.append(temp)
        
        return L
    
                
    def make_set(self, itemset): # 재귀함수 or from itertools import combinations
        aa = list(itemset)
        n = len(aa)
        L = []
        
        for i in range(1<<n):
            s = set([])
            for j in range(n):
                t_f = i & (1 <<j)
                if t_f != 0:
                    s.add(int(aa[j]))
            L.append(s)
        
        L.pop()
        L.pop(0)
    
        return L

   
    def cal_sup_conf(self, i, db):
        i_len = len(i)
        db_len = len(db)
        dic = {}
        
        for j in i:
            count = 0
            count_2 = 0
            temp = j[0].union(j[1])
            for k in range(db_len):
                if temp.issubset(db[k]) == True:
                    count = count +1
                if j[0].issubset(db[k]) == True:
                    count_2 = count_2 +1
            b = i.index(j)
            dic[b] = [count, count_2]

        for c,j in dic.items():    
            sup = '%.2f'%round(100*j[0]/db_len,2)
            conf = '%.2f'%round(100*j[0]/j[1],2)
            if len(i[c]) >= 4:
                pass
            else:
                i[c].append(sup)
                i[c].append(conf)
        
        return i
    
    def make_output(self, j, output):
        f = open(output, 'w')
        
        for i in j:
            for k in i[0:2]:
                temp_2 = 1
                f.write("{")
                for p in k:
                    t = (str(p))
                    f.write(t)
                    if temp_2 == len(k):
                        f.write("}\t")
                    else:
                        f.write(",")
                        temp_2 += 1

            for k in i[2:]:
                t = str(k)
                f.write(t)
                f.write("\t")
            f.write("\n")
        f.close()        

In [ ]:
a = Algorithm(5)
b = a.Partition_Database(List, 5)
f = a.Scan_one(b)
g = a.Consolidate(f)
h = a.Scan_two(g, List)
i = a.association_rule(h)
j = a.cal_sup_conf(i, List)
k = a.make_output(j, "output.txt")

In [ ]:
# 딕셔너리
# defaultdict

# 중복요소 카운팅
# for i in lists:
#     try: count[i] += 1
#     except: count[i] = 1

# 리스트와 set
# int str float
# 소숫점아래 몇자리 '%.2f'%num / 반올림 round(2.5,2)

# map함수
# originalList = ['2', '4', '6', '8', '10', '12']
# intList = list(map(int, originalList))
# print(intList)
# [2, 4, 6, 8, 10, 12]

# 다이나믹 프로그래밍 / 동적 프로그래밍

# import pandas as pd
# from collections import defaultdict
# import time